In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#
import pandas as pd
from io import StringIO
#
import logging

In [ ]:
class CIWQS:
    def __init__(self):
        self.driver = webdriver.Firefox()#options=options) # not working
        self.logger = logging.getLogger('ciwqs')
        
    def __del__(self):
        self.driver.quit()
        
    def download_npdes_permit_pdf(self, placeid):
        url = f'https://ciwqs.waterboards.ca.gov/ciwqs/readOnly/CiwqsReportServlet?inCommand=drilldown&reportName=facilityAtAGlance&placeID={placeid}'
        self.driver.get(url)
        self.logger.debug(self.driver.find_element_by_class_name('ciwqsReportDataTable').text)
        tables = self.driver.find_elements_by_class_name('ciwqsReportDataTable')
        self.logger.debug(f'Length of tables: {len(tables)}')
        self.logger.debug(tables[3].text.find('Regulatory Measures') >= 0)
        df = pd.read_html(StringIO(tables[3].get_attribute('innerHTML')), header=0)[0]
        self.logger.debug(df)
        #reg_measure_id = df.query('Status == "Active"')['Reg Measure ID'].values[0]
        reg_measure_id = df.query('`Reg Measure Type` == "NPDES Permit"')['Reg Measure ID'].values[0]
        report_link = tables[3].find_element_by_xpath(f"//tr//*[contains(text(),{reg_measure_id})]").find_element_by_xpath('../..').find_element_by_tag_name("a")#.get_attribute('href')
        self.logger.debug(f"link to pdf reports: {report_link.get_attribute('href')}")
        self.driver.get(report_link.get_attribute('href'))
        self.logger.debug(self.driver.find_element_by_class_name('ciwqsReportDataTable').text)
        self.logger.debug(self.driver.find_element_by_partial_link_text(".pdf").get_attribute('href'))
        pdf_report_links = self.driver.find_elements_by_partial_link_text(".pdf")
        fnames = []
        for pdf_report_link in pdf_report_links:
            fnames.append(pdf_report_link.text)
            self.logger.info('Downloading PDF', pdf_report_link.text)
            pdf_report_link.click()
        return fnames

In [ ]:
c = CIWQS()

In [ ]:
facility_id = pd.read_csv('facility_name_id.csv',index_col=0)

In [ ]:
facility_id.query('facility_name.str.contains("Sacramento")')

In [ ]:
import tqdm

In [ ]:
def download_filenames(place_id):
    try:
        return c.download_npdes_permit_pdf(place_id)
    except Exception as e:
        #raise e
        return []

In [ ]:
place_id_filenames_map = {r['facility_place_id']: download_filenames(r['facility_place_id']) for i, r in tqdm.tqdm(facility_id.iterrows(),total=len(facility_id))}

In [ ]:
import pickle

In [ ]:
with open('place_id_filenames_map.pkl','wb') as fh: pickle.dump(place_id_filenames_map, fh)

In [ ]:
del c